# Mapping the ocean floor

Here's hoping that my games programming past comes back to help me here.

First of all, we're going to want to examine the floor.  The critical bit of information we need is that for any given pair of coordinates, (x,y) we should be able to ask it for the heights of its neighbours, which will return a list like `[1,1,2,3]` or `[1,1]`.  If we ask an edge or a corner for its neighbours, we'll just return the 2 or 3 neighbours.

Once we have that, we can then iterate over all the neighbours for each cell and ask if they are all greater than the current cell, if they are, we've found a "local minima".

In [2]:
## Import ipytest and get it setup for use in Python Notebook
import pytest
import ipytest
ipytest.autoconfig()

In [3]:
class Grid:
    def __init__(self,width,height,cells):
        self.width = width
        self.height = height
        self.cells = cells
        
    def find_neighbours(self, x,y):
        neighbours = []
        if y > 0:
            neighbours.append(self.cells[(x,y-1)])
        if x < self.width-1:
            neighbours.append(self.cells[(x+1,y)])
        if y < self.height-1:
            neighbours.append(self.cells[(x,y+1)])
        if x > 0:
            neighbours.append(self.cells[(x-1,y)])
        return neighbours

def grid_from_lines(lines):
    height = len(lines)
    width = len(lines[0])
    cells = {}
    for y,line in enumerate(lines):
        for x,cell in enumerate(line):
            cells[(x,y)] = int(cell)
    return Grid(width,height,cells)

test_data = """2199943210
3987894921
9856789892
8767896789
9899965678""".split("\n")

test_grid = grid_from_lines(test_data)
assert 10 == test_grid.width
assert 5 == test_grid.height
assert 2 == test_grid.cells[(0,0)]
assert 1 == test_grid.cells[(1,0)]
assert 3 == test_grid.cells[(0,1)]

# Implement find_neighbours on Grid class here

assert [1,1] == test_grid.find_neighbours(9,0)
assert [9,9,2] == test_grid.find_neighbours(1,0)
assert [8,6,6,8] == test_grid.find_neighbours(2,2)

Right, now we've got a basic grid class, we can find the lowest point.

Sadly, due to the way the Jupyter notebook works, we cannot reopen an existing class, so we have to redefine it if we want to add a method, which we do in this case.  Note that we're interested in the heights of the lowest points.

We could do this two ways, the longer and more readable way is to make find_lowest_points return a list of coordinates, and then write something that maps those into heights and then adds one to each one to get the risk.  Or we can put that into the Grid class as a find_risk_score.  I'm going to do the latter for speed, but I think we might need to come back and create a find_lowest_points later.

In [4]:
class Grid:
    def __init__(self,width,height,cells):
        self.width = width
        self.height = height
        self.cells = cells
        
    def find_neighbours(self, x,y):
        neighbours = []
        if y > 0:
            neighbours.append(self.cells[(x,y-1)])
        if x < self.width-1:
            neighbours.append(self.cells[(x+1,y)])
        if y < self.height-1:
            neighbours.append(self.cells[(x,y+1)])
        if x > 0:
            neighbours.append(self.cells[(x-1,y)])
        return neighbours
    
    def find_risk_score(self):
        score = 0
        for y in range(self.height):
            for x in range(self.width):
                if self.cells[(x,y)] < min(self.find_neighbours(x,y)):
                    score += self.cells[(x,y)]+1
        return score

def grid_from_lines(lines):
    height = len(lines)
    width = len(lines[0])
    cells = {}
    for y,line in enumerate(lines):
        for x,cell in enumerate(line):
            cells[(x,y)] = int(cell)
    return Grid(width,height,cells)

test_grid = grid_from_lines(test_data)

assert 15 == test_grid.find_risk_score()

Great!  Let's try that on prod data

In [5]:
grid = grid_from_lines([line.strip() for line in open('day9.txt')])
print(grid.find_risk_score())

572


## Part 2 Filling the basin

I knew I'd need the find_lowest_points.

This is described in a way that makes it look hard, but in fact it's a fairly simple search algorithm.

In this case, what they didn't say out loud, but is true when you look at the examples is that basins are enclosed on all sides by either the edge of the map, or by 9's.

So what we need to do is calculate a basin score for each lowest point, and then we can order them, take the top 3 and multiply them.

How do we calculate a basin score?  Well, we can use a simple open and closed list for coordinates, and then explore.
When we explore, we start with the first item from the open list, we put it on teh closed list, and then we look at it's neighbours.  If they are less than 9, we put them on the open list.  If they are 9, they get ignored.  If they are on the closed list they are ignored, if they are already on the open list they are ignored.

Then we keep doing that until we run out of coordinates on the open list.

Let's describe that with a really simple basin

```929
919
999```

We start at 1,1, which is our lowest point, so open = [(1,1)] and closed is [].
We put (1,1) on the closed list and remove it from the open list, resulting in open=[], closed=[(1,1)]
We evaluate the neighbours of (1,1), (1,0) is 2, so we add it to the open list, (2,1) is a 9, so we skip it, (1,2) is a 9 so we skip it, and (0,1) is a 9 so we skip it.  We now have just (1,0) on the open list.

We pop (1,0) on the closed list and remove it from the open list.
We examine it's neighbours, (2,0) is a 9, ignore it.  (1,1) is on the closed list, ignore it.  (0,1) is a 9, ignore it.

Our open list is now empty, so we return, and the closed list contains all of our basin nodes.

In [6]:
import operator

class Grid:
    def __init__(self,width,height,cells):
        self.width = width
        self.height = height
        self.cells = cells
        
    def get_cell(self, coord):
        return self.cells[coord]
        
    def find_neighbours(self, x,y):
        neighbours = []
        if y > 0:
            neighbours.append((x,y-1))
        if x < self.width-1:
            neighbours.append((x+1,y))
        if y < self.height-1:
            neighbours.append((x,y+1))
        if x > 0:
            neighbours.append((x-1,y))
        return neighbours
    
    def find_risk_score(self):
        return sum(map(lambda c: self.cells[(c[0],c[1])]+1, self.find_lowest_points()))
    
    def find_lowest_points(self):
        return [(x,y) for x in range(self.width) for y in range(self.height) if self.cells[(x,y)] < min(map(self.get_cell, self.find_neighbours(x,y)))]
    
    def find_basins(self):
        basins = []
        for point in self.find_lowest_points():
            open = [point]
            closed = []
            while open:
                point = open.pop()
                closed.append(point)
                for neighbour in self.find_neighbours(*point):
                    if neighbour not in closed and neighbour not in open and self.cells[neighbour] != 9:
                        open.append(neighbour)
            basins.append(closed)
        return basins
    
    def display(self, highlights=[]):
        import IPython.display
        IPython.display.display("**************************************")
        for y in range(self.height):
            line = ""
            for x in range(self.width):
                if (x,y) in highlights:
                    line += f" **{self.get_cell((x,y))}** "
                else:
                    line += f" {self.get_cell((x,y))} "
            IPython.display.display(IPython.display.Markdown(line))   
        IPython.display.display("**************************************")
        
def grid_from_lines(lines):
    height = len(lines)
    width = len(lines[0])
    cells = {}
    for y,line in enumerate(lines):
        for x,cell in enumerate(line):
            cells[(x,y)] = int(cell)
    return Grid(width,height,cells)

test_grid = grid_from_lines(test_data)
test_grid.display()

assert 15 == test_grid.find_risk_score()

test_basins = test_grid.find_basins()
for basin in test_basins:
    test_grid.display(basin)
    print(len(basin))

    assert 4 == len(test_basins)
assert sorted([(0,0),(1,0),(0,1)]) == sorted(test_basins[0])
test_basin_sizes = sorted(map(len, test_basins))
assert [3,9,9,14] == test_basin_sizes
assert 1134 == test_basin_sizes[-3]*test_basin_sizes[-2]*test_basin_sizes[-1]

'**************************************'

 2  1  9  9  9  4  3  2  1  0 

 3  9  8  7  8  9  4  9  2  1 

 9  8  5  6  7  8  9  8  9  2 

 8  7  6  7  8  9  6  7  8  9 

...

 9  8  5  6  7  8  9  8  9  **2** 

 8  7  6  7  8  9  6  7  8  9 

 9  8  9  9  9  6  5  6  7  8 

'**************************************'

9


That works, let's do this

In [7]:
grid = grid_from_lines([line.strip() for line in open('day9.txt')])
basins = grid.find_basins()
basin_sizes = sorted(map(len, basins))
print(basin_sizes[-3]*basin_sizes[-2]*basin_sizes[-1])

847044


For fun, I'd added a printing method, so let's print out the production basic.  What would be nice would be if I could work out how to get Python notebook to render this as markdown, but I've got work to do today... maybe at lunch

In [8]:
grid.display([coord for basin in basins for coord in basin])

'**************************************'

 **5**  **7**  9  **6**  **7**  9  **8**  **6**  **2**  **1**  **2**  **3**  **7**  9  9  **5**  **4**  9  **8**  **7**  **6**  **5**  **4**  **3**  **4**  **5**  **6**  **7**  9  **8**  **7**  **5**  **4**  **2**  9  9  9  **7**  **6**  **5**  **6**  **7**  9  9  **8**  **7**  **5**  **4**  **5**  **6**  **7**  9  **1**  **0**  9  **8**  **7**  **8**  9  9  9  **8**  **7**  **7**  **8**  9  9  **7**  **8**  9  **8**  **7**  **6**  **5**  **3**  **2**  **1**  **2**  **3**  **4**  **5**  **6**  9  9  **8**  9  9  9  **8**  **7**  **6**  **8**  **8**  **7**  **8**  9  9  9  **2**  **1** 

 **4**  **6**  **4**  **5**  9  **7**  **6**  **4**  **3**  **4**  **4**  **5**  **6**  **7**  **8**  9  **3**  **4**  9  **6**  **5**  **4**  **3**  **2**  **1**  **2**  9  **8**  9  9  **7**  **6**  **7**  9  **8**  9  **8**  **6**  **5**  **4**  **6**  9  **8**  9  **8**  **7**  **6**  **3**  **5**  **6**  **7**  **8**  9  9  **8**  **7**  **6**  **7**  **8**  9  **7**  **6**  **5**  **6**  **7**  **8**  9  **6**  9  **8**  **7**  **6**  **5**  **4**  **3**  **2**  **0**  **1**  **2**  **3**  **4**  **7**  **8**  9  **7**  **8**  9  9  **8**  **6**  **5**  **6**  **7**  **6**  **7**  9  **8**  **7**  9  9 

 **3**  **2**  **3**  **4**  9  **8**  **7**  **5**  **4**  **5**  9  **7**  **8**  9  9  **3**  **2**  9  **8**  **7**  9  **5**  **4**  **1**  **0**  9  **8**  9  9  9  **8**  9  **8**  9  **7**  **8**  9  **5**  **4**  **3**  **2**  **5**  **6**  **8**  9  **7**  **6**  **4**  **6**  **7**  **8**  9  **4**  9  **8**  **7**  **5**  **6**  **7**  **8**  9  **5**  **4**  **6**  **8**  9  **6**  **5**  **7**  9  **8**  **7**  **7**  **8**  **4**  **2**  **1**  **2**  **3**  **4**  **5**  **6**  9  **7**  **6**  **7**  **8**  9  9  **5**  **4**  **3**  **4**  **5**  9  **8**  **5**  **6**  **7**  **8** 

 **4**  **3**  **5**  **6**  **7**  9  **8**  **6**  **7**  9  9  **8**  9  9  9  9  **0**  **1**  9  9  **8**  **7**  **3**  **2**  9  **8**  **7**  **8**  9  9  9  **7**  9  **8**  **6**  **7**  9  **7**  **6**  **5**  **3**  **4**  **5**  **6**  **8**  9  **8**  **5**  **6**  9  9  **1**  **2**  9  **6**  **5**  **4**  **5**  **8**  9  **0**  **1**  **3**  **4**  **7**  **8**  9  **4**  **6**  **7**  9  9  **8**  9  **5**  **6**  **8**  9  9  9  **6**  **7**  **8**  9  **5**  **6**  9  **8**  **6**  **4**  **3**  **2**  **3**  9  **8**  **7**  **4**  **5**  **7**  9 

...

 **3**  **7**  **7**  9  9  9  **2**  9  **7**  **6**  **5**  **5**  **3**  **2**  **3**  **5**  **6**  **7**  9  9  **7**  **7**  **4**  **5**  **5**  **6**  9  **8**  9  9  **8**  **7**  **6**  **5**  **4**  **8**  **7**  9  **8**  **7**  **6**  **8**  9  **8**  **6**  **4**  **3**  **4**  **3**  **4**  **5**  **8**  9  9  9  **8**  **7**  **6**  **5**  **4**  **3**  **2**  **3**  **7**  **8**  9  **7**  **5**  **6**  9  9  **8**  9  **7**  9  **7**  **6**  **5**  **2**  **1**  **0**  **1**  **4**  **5**  9  **8**  **7**  **6**  **5**  **4**  **3**  **4**  **5**  **6**  **7**  **8**  9  **7**  **8**  9 

 9  **8**  **8**  9  **6**  **8**  9  9  **8**  **7**  **6**  **6**  **4**  **3**  **4**  **6**  **7**  9  **8**  **7**  **6**  **5**  **3**  **2**  **3**  **4**  **5**  **6**  **7**  **8**  9  **8**  **8**  **7**  **5**  **6**  9  **8**  **7**  **6**  **5**  9  **8**  **7**  **6**  **5**  **4**  **5**  **4**  **5**  **6**  9  9  **8**  **6**  9  **8**  **7**  **6**  **5**  **2**  **1**  **4**  **5**  **7**  **8**  9  **6**  9  **8**  **7**  9  **5**  **6**  9  **8**  **7**  **6**  **8**  **8**  9  **2**  **3**  **6**  **8**  9  **8**  **7**  9  **5**  **6**  **5**  **6**  **7**  **8**  9  **5**  **6**  **7**  **8** 

 **4**  9  9  **4**  **5**  **6**  **7**  **8**  9  9  **8**  **6**  **5**  **4**  **5**  **7**  9  9  9  9  **7**  **4**  **3**  **1**  **3**  **3**  **4**  **5**  **6**  9  9  9  9  **8**  **7**  9  **8**  **7**  **6**  **5**  **4**  **3**  9  **8**  **7**  9  **8**  **7**  **5**  **6**  **7**  **8**  9  **5**  **5**  **6**  9  **8**  **7**  **6**  **7**  **2**  **3**  **4**  9  9  9  9  **8**  **7**  **6**  **5**  **4**  **2**  **2**  9  **8**  **7**  **8**  **7**  **6**  **5**  **4**  **5**  **6**  **7**  9  9  **7**  **6**  **7**  **8**  **7**  **8**  9  **3**  **4**  **5**  **8**  9 

 **2**  **1**  **2**  **3**  **4**  **5**  **6**  9  9  9  **8**  **7**  **6**  **5**  **6**  **7**  **8**  9  9  **8**  **7**  **5**  **1**  **0**  **1**  **2**  **5**  **6**  **7**  **8**  9  **1**  **0**  9  9  9  9  9  **8**  **4**  **3**  **2**  **1**  9  **8**  9  9  **8**  9  **8**  9  9  **5**  **4**  **3**  **2**  **4**  9  **8**  **7**  **8**  9  **4**  **5**  **6**  **7**  **8**  9  9  **6**  **5**  **4**  **2**  **1**  **0**  **1**  9  **8**  9  **8**  **7**  **7**  **6**  **6**  **7**  **8**  9  9  **8**  **7**  **8**  9  **8**  9  **1**  **2**  **3**  **6**  9  9 

'**************************************'